This is a simplified example of building a RAG system using OpenAI's LLM embeddings without relying on an external framework. You can extend and customize this code based on your specific requirements and use case.

In [54]:
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import os

In [55]:
# Set up OpenAI API credentials
client = OpenAI(api_key = os.environ['OPENAI_API_KEY'])

In [56]:
# Define the knowledge base
knowledge_base = [
    "In the year 2045, a young scientist named Lila Jacobson invented a revolutionary AI system called \"Neurolink\" that could directly interface with the human brain.",
    "The planet Zephyria, located in the Andromeda galaxy, is home to a race of intelligent beings called the Zephyrians, who have mastered intergalactic teleportation.",
    "In an alternate timeline, the ancient Egyptians discovered a mysterious crystal that granted them advanced technological abilities, allowing them to build the pyramids using anti-gravity technology.",
    "The legendary city of Atlantis was not lost beneath the waves but instead was a floating city that could traverse the oceans using advanced hydrodynamic engines.",
    "In the year 2120, humans discovered a way to upload their consciousness into robotic bodies, achieving a form of digital immortality.",
    "The Illumirians, a secret society of magic wielders, have been guiding the course of human history from the shadows for millennia.",
    "In a parallel universe, humans evolved to possess telepathic abilities, leading to the development of a global hive-mind consciousness.",
    "The Crimson King, a powerful sorcerer, once ruled over the mystical land of Eldritch, where magic and technology were indistinguishable.",
    "In the 31st century, humans have colonized the entire solar system and have begun to terraform planets in nearby star systems.",
    "The Chrono Guardians, a group of time-travelers, work tirelessly to maintain the integrity of the timeline and prevent temporal paradoxes."
]

In [57]:
# Generate embeddings for the knowledge base
def get_embeddings(texts):
    response = client.embeddings.create(
        input=texts,
        model="text-embedding-3-small"
    )
    return [r.embedding for r in response.data]

kb_embeddings = get_embeddings(knowledge_base)

In [58]:
# Function to retrieve relevant information from the knowledge base
def retrieve_info(query, top_k=3):
    query_embedding = get_embeddings([query])[0]
    similarities = cosine_similarity([query_embedding], kb_embeddings)[0]
    top_indices = np.argsort(similarities)[-top_k:][::-1]
    return [knowledge_base[i] for i in top_indices]

In [59]:
# Function to generate a response using OpenAI's language model
def generate_response(prompt):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100,
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].message.content

In [60]:
# Main function to handle user queries
def handle_query(query):
    relevant_info = retrieve_info(query)
    prompt = f"Relevant information:\n{' '.join(relevant_info)}\n\nQuery: {query}\n\nResponse:"
    response = generate_response(prompt)
    return response

In [69]:
# Example usages
queries = [
    "In what year did Lila Jacobson invent the \"Neurolink\" AI system?",
    "What is the name of the intelligent beings who live on the planet Zephyria?",
    "Who works to maintain the integrity of the timeline and prevent temporal paradoxes?",
    "Who ruled over the mystical land of Eldritch?",
    "In what century have humans colonized the entire solar system?"
]

for query in queries:
    response = handle_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}\n")

Query: In what year did Lila Jacobson invent the "Neurolink" AI system?
Response: Lila Jacobson invented the "Neurolink" AI system in the year 2045.

Query: What is the name of the intelligent beings who live on the planet Zephyria?
Response: The intelligent beings who live on the planet Zephyria are called the Zephyrians.

Query: Who works to maintain the integrity of the timeline and prevent temporal paradoxes?
Response: The Chrono Guardians are the group of time-travelers who work tirelessly to maintain the integrity of the timeline and prevent temporal paradoxes. Their role is crucial in ensuring that the past, present, and future remain stable and free from disruptions caused by time-travel activities.

Query: Who ruled over the mystical land of Eldritch?
Response: The Crimson King, a powerful sorcerer, ruled over the mystical land of Eldritch.

Query: In what century have humans colonized the entire solar system?
Response: Humans have colonized the entire solar system in the 31st